In [3]:
import pandas as pd

# Загрузка обучающих данных из JSON
train_df = pd.read_json('./methodcompetition1/train.json')

# Отображение первых нескольких строк обучающего набора данных
train_df.head()

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Функция предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление пунктуации
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Склеивание токенов обратно в текст
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

# Применение функции предобработки к столбцу с текстом
train_df['preprocessed_text'] = train_df['text'].apply(preprocess_text)

# Отображение первых нескольких строк с предобработанным текстом
train_df[['text', 'preprocessed_text']].head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Evgenie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Evgenie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Evgenie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,preprocessed_text
0,Досудебное расследование по факту покупки ЕНПФ...,досудебное расследование факту покупки енпф па...
1,Медики рассказали о состоянии пострадавшего му...,медики рассказали состоянии пострадавшего мужч...
2,"Прошел почти год, как железнодорожным оператор...",прошел год железнодорожным операторам запретил...
3,По итогам 12 месяцев 2016 года на территории р...,итогам 12 месяцев 2016 года территории республ...
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,астана 21 ноября kazakhstan today агентство рк...


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(train_df['preprocessed_text'], train_df['sentiment'], test_size=0.2, random_state=42)

# Преобразование текстовых данных в векторы признаков с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели LinearSVC
model = LinearSVC()
model.fit(X_train_tfidf, y_train)

# Предсказание на тестовом наборе данных
y_pred = model.predict(X_test_tfidf)

# Оценка производительности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.6963097398669087
Classification Report:
              precision    recall  f1-score   support

    negative       0.72      0.61      0.66       285
     neutral       0.71      0.75      0.73       830
    positive       0.67      0.66      0.66       538

    accuracy                           0.70      1653
   macro avg       0.70      0.67      0.68      1653
weighted avg       0.70      0.70      0.70      1653



C:\Users\Evgenie\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [8]:
from joblib import dump

# Сохранение tfidf_vectorizer в файл
dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']

In [7]:
from joblib import dump

# Сохранение модели в файл
dump(model, 'sentiment_classifier.joblib')

['sentiment_classifier.joblib']